In [1]:
# pip install geopandas

In [2]:
import geopandas as gpd
import pandas as pd
import os

In [3]:
os.getcwd()

'/Users/hayley/Jupyterlab'

In [4]:
hosp_data = pd.read_csv('clinics_Cali.csv')
hosp_data = hosp_data[hosp_data['match']].copy()
hosp_data.reset_index(inplace=True, drop=True)
hosp_data.head(2)

,NPI,Provider Organization Name (Legal Business Name),Provider First Line Business Practice Location Address,Provider Business Practice Location Address Telephone Number,Address,City,State2,Zip5,address,match,lat,lon
0,1548468614,DIMENSION PROSTHETICS & ORTHOTICS,33374 DOWE AVE,5103243400,33374 DOWE AVE,UNION CITY,CA,94587,"33374 DOWE AVE, UNION CITY, CA, 94587",True,37.601059,-122.043522
1,1952507303,"SOUND BALANCE AUDIOLOGY, INC",2420 VISTA WAY,7607217417,2420 VISTA WAY,OCEANSIDE,CA,92054,"2420 VISTA WAY, OCEANSIDE, CA, 92054",True,33.183241,-117.334400


In [5]:
hosp_geo = gpd.GeoDataFrame(hosp_data, geometry=gpd.points_from_xy(hosp_data.lon, hosp_data.lat), crs="EPSG:4326")

In [6]:
cali_counties = gpd.read_file(r'tl_2016_06_cousub/tl_2016_06_cousub.shp')
cali_outline = cali_counties.dissolve('STATEFP')
cali_proj = cali_outline.to_crs('EPSG:5070')
print(cali_outline.crs)

EPSG:4269


In [7]:
def dissolve_buff(point_df,d,resolution):
    bu = point_df.buffer(d,resolution)
    geodf = gpd.GeoDataFrame(geometry=bu)
    geodf['Const'] = 0
    single = geodf.dissolve('Const')
    return single[['geometry']]

In [8]:
def dist_cont(point_df,dist_list,outside,buff_res):
    if point_df.crs != outside.crs:
        print('Point df and Outside df are not the same CRS')
        return None
    # Making outside area out dissolved object
    out_cop = outside[['geometry']].copy()
    out_cop['Constant'] = 1
    out_cop = out_cop.dissolve('Constant')
    # Make sure points are inside area
    inside = point_df.within(out_cop['geometry'][1])
    point_cop = point_df[inside].copy()
    point_cop = point_df.copy()
    point_cop['Constant'] = 1 #Constant for dissolve
    point_cop = point_cop[['Constant','geometry']].copy()
    res_buffers = []
    for i,d in enumerate(dist_list):
        print(f'Doing buffer {d}')
        if i == 0:
            res = dissolve_buff(point_cop, d, buff_res)
            res_buffers.append(res.copy())
        else:
            res_new = dissolve_buff(point_cop, d, buff_res)
            res_buffonly = gpd.overlay(res_new, res, how='difference')
            res = res_new.copy()
            res_buffers.append( res_buffonly.copy() )
    # Now take the difference with the larger area
    print('Working on leftover difference now')
    leftover = gpd.overlay(out_cop, res, how='difference')
    res_buffers.append(leftover)
    for i,d in enumerate(dist_list):
        res_buffers[i]['Distance'] = str(d)
    res_buffers[-1]['Distance'] = 'Outside'
    # New geopandas DF
    comb_df = pd.concat(res_buffers)
    comb_df.reset_index(inplace=True, drop=True)
    return comb_df

In [9]:
hos_proj = hosp_geo.to_crs('EPSG:5070') #'epsg:4269'

dist_met = [2000, 4000, 8000, 16000] #, 32000
buff_city = dist_cont(hos_proj, dist_met, cali_proj, buff_res=100)

Doing buffer 2000
Doing buffer 4000
Doing buffer 8000
Doing buffer 16000
Working on leftover difference now


In [10]:
#Now making folium plot
buff_map = buff_city.to_crs('EPSG:4326')
kv = list(hosp_geo)[1:10]

In [ ]:
#"fill": "#00aa22",
#"fill-opacity": 0.5

cols = ['#f1eef6',
'#d7b5d8',
'#df65b0',
'#dd1c77',
'#980043']

buff_map['fill'] = cols
buff_map['fill-opacity'] = 0.35

#os.chdir(r'D:\Dropbox\Dropbox\PublicCode_Git\Blog_Code')

buff_map.to_file('Buffers.geojson', driver='GeoJSON')
hosp_geo.to_file('Hosp.geojson', driver='GeoJSON')